In [1]:
####----Check correct working directory----####

import os
wd = os.getcwd()
if(wd != '/home/centos/BigData/02_cleaning/'):
    from os import chdir
    chdir("/home/centos/BigData/02_cleaning/")
    print("Directory changed to: " + wd)
else:
    print("Correct current directory: " + wd)

Directory changed to: /home/centos/BigData/02_cleaning


In [2]:
####----Import dictionary----####
with open('dict/Overall_Master.txt', encoding = 'ISO-8859-1') as file:
    ingredient_set = file.read().splitlines()

print("List length: " +str(len(ingredient_set)))
#print(ingredient_set)
#ingredient_set = ingredient_set[1:10]

List length: 525


In [4]:
####----Connect to Spark via PySpark----####
%pip install pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode, col, lit, monotonically_increasing_id
from pyspark.sql.types import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Normalizer, CountVectorizer
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
print("Spark connected")

#spark.close()
#spark.stop()


#spark = SparkContext(conf = SparkConf().setAppName("PySpark"))

spark = SparkSession \
    .builder \
    .appName("Python Spark") \
    .config("spark.some.config.option", "value") \
    .getOrCreate()
print("Spark session created")

##Python Java issues: a drawback in using pyspark instead of Scala



Note: you may need to restart the kernel to use updated packages.
Spark connected
Spark session created


In [5]:
# %pip install itertools
# from itertools import compress

#%pip install json
#import json

In [7]:
####----Load json into Spark for data cleaning----####
read_path = "/home/centos/BigData/01_source/allrecipes*.json"
#read_path = "hdfs://localhost:9000/01_raw/allrecipes*.json"

recipe_j = spark.read.json(read_path)
recipe_j.printSchema()

recipe_j.head(1)

root
 |-- author: string (nullable = true)
 |-- cook_time_minutes: long (nullable = true)
 |-- description: string (nullable = true)
 |-- error: boolean (nullable = true)
 |-- footnotes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ingredients: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- instructions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- photo_url: string (nullable = true)
 |-- prep_time_minutes: long (nullable = true)
 |-- rating_stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- time_scraped: long (nullable = true)
 |-- title: string (nullable = true)
 |-- total_time_minutes: long (nullable = true)
 |-- url: string (nullable = true)



[Row(author='Stephanie', cook_time_minutes=25, description='I just started adding my favorite things to basic cornbread and I came up with something great!', error=False, footnotes=[], ingredients=['1/2 cup unsalted butter, chilled and cubed', '1 cup chopped onion', '1 3/4 cups cornmeal', '1 1/4 cups all-purpose flour', '1/4 cup white sugar', '1 tablespoon baking powder', '1 1/2 teaspoons salt', '1/2 teaspoon baking soda', '1 1/2 cups buttermilk', '3 eggs', '1 1/2 cups shredded pepperjack cheese', '1 1/3 cups frozen corn kernels, thawed and drained', '2 ounces roasted marinated red bell peppers, drained and chopped', '1/2 cup chopped fresh basil'], instructions=['Preheat oven to 400 degrees F (205 degrees C). Butter a 9x9x2 inch baking pan.', 'Melt 1 tablespoon butter in medium nonstick skillet over medium-low heat. Add onion and saute until tender, about 10 minutes. Cool.', 'Mix cornmeal with the flour, baking powder, sugar, salt, and baking soda in large bowl. Add 7 tablespoons butte

In [8]:
####----Create Spark DataFrame----####

##Create temporary view to transform to dataframes
recipe_j.createOrReplaceTempView("recipe")

##Create Spark dataframe
recipe_df = spark.sql("SELECT title, description, ingredients, instructions, rating_stars, review_count, photo_url, total_time_minutes, url FROM recipe LIMIT 1000")
recipe_df.printSchema()

recipe_df.show(5)

##Generate into sample
# small_sample = recipe_sdf.toPandas()
# small_sample.to_csv("small_sample.csv")

#Noted cook_time_minutes have quite a bit of zeroes. Use rating_stars and review_count instead


root
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- instructions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rating_stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- photo_url: string (nullable = true)
 |-- total_time_minutes: long (nullable = true)
 |-- url: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+------------+------------+--------------------+------------------+--------------------+
|               title|         description|         ingredients|        instructions|rating_stars|review_count|           photo_url|total_time_minutes|                 url|
+--------------------+--------------------+--------------------+--------------------+------------+------------+--------------------+------------------+--------------------+
|Bas

In [9]:
####----Extract ingredient----####
import re

##User Defined Functions
def removeParenthesis(li):
    output = []
    for text in li:
        text = re.sub(r"\([^)]*\)", "", text)
        text = text.strip()
        output.append(text)
    return output


def extractIngredient(li, ingredient_set):
    output = []
    for item in li:
        temp_output = []
        for set_tracker in range(len(ingredient_set)):
            check = bool(re.search(ingredient_set[set_tracker], item))
            if (check == True):    
                temp_output.append(ingredient_set[set_tracker])
        if (len(temp_output) != 0):
##Comment
#             temp_counter = 0
#             temp_tracker = 0
#             for temp in range(len(temp_output)):
#                 count_temp = len(temp_output[temp])
#                 if (count_temp > temp_counter):
#                     temp_tracker = temp
#                     temp_counter = count_temp
#             output.append(temp_output[temp_tracker])
##Comment
            output.append(temp_output[[len(i) for i in temp_output].index(max([len(i) for i in temp_output]))])
    return output


##Testing:
#sample = ['1/2 cup unsalted butter, chilled and cubed', '1 cup chopped onion', '1 3/4 cups cornmeal', '1 1/4 cups all-purpose flour', '1/4 cup white sugar', '1 tablespoon baking powder', '1 1/2 teaspoons salt', '1/2 teaspoon baking soda', '1 1/2 cups buttermilk', '3 eggs', '1 1/2 cups shredded pepperjack cheese', '1 1/3 cups frozen corn kernels, thawed and drained', '2 ounces roasted marinated red bell peppers, drained and chopped', '1/2 cup chopped fresh basil']
#test = extractIngredient(sample, ingredient_set)
#print(test)


##Create User Defined Function
udf_removeParen = udf(removeParenthesis, ArrayType(StringType()))
udf_extractIngredient = udf(lambda x: extractIngredient(x,ingredient_set), ArrayType(StringType()))
udf_countList = udf(lambda x: len(x), IntegerType())

print("UDF successful")

UDF successful


In [15]:
# ####----Parallel processing----####
# %pip install multiprocessing
# import multiprocessing as mp

# #Define an output queue
# output = mp.Queue()


# def cleaning():{
# ####----Create ingredient table----####
# ##Create ingredient table
# recipe_j.select(["title", "ingredients"])
# #ingredient_tmp_df = recipe_df.select(["title", "ingredients"])

# ingredient_tmp_df.withColumn('rm_paren', udf_removeParen('ingredients')).select('title','ingredients','rm_paren') \
#                          .withColumn('ingredient_extract', udf_extractIngredient('rm_paren')).select('title','ingredient_extract') \
#                          .withColumn('extract_count', udf_countList('ingredient_extract')).select('title','ingredient_extract', 'extract_count')
# return ingredient_df
# }
    

    
    
# ##Setup a list of processes that we want to run
# processes = [mp.Process(target = cleaning)]


# ingredient_df.show(5)

# ##Run processes
# for p in processes:
#     p.start()

# ##Exit the completed processes
# for p in processes:
#     p.join()

# ##Get process results from the output queue
# results = [output.get() for p in processes]
# results.sort()
# results = [r[1] for r in results]
# print(results)

# pool = mp.Pool(processes=4)
# results = [pool.apply(cleaning)]


In [9]:
####----Create ingredient table----####
##Create ingredient table
ingredient_tmp_df = recipe_j.select(["title", "ingredients"])
#ingredient_tmp_df = recipe_df.select(["title", "ingredients"])


ingredient_df = ingredient_tmp_df.withColumn('rm_paren', udf_removeParen('ingredients')).select('title','ingredients','rm_paren') \
                         .withColumn('ingredient_extract', udf_extractIngredient('rm_paren')).select('title','ingredient_extract') \
                         .withColumn('extract_count', udf_countList('ingredient_extract')).select('title','ingredient_extract', 'extract_count')
ingredient_df.show(5)


+--------------------+--------------------+-------------+
|               title|  ingredient_extract|extract_count|
+--------------------+--------------------+-------------+
|Basil, Roasted Pe...|[butter, onion, c...|           14|
|Crispy Cheese Twists|[cheese, pepper, ...|            5|
|   Mom's Yeast Rolls|[water, margarine...|            8|
|Sweet Potato Bread I|[sugar, vegetable...|           11|
|         Orange Buns|[butter, sugar, m...|           10|
+--------------------+--------------------+-------------+
only showing top 5 rows



In [ ]:
##Convert into list form for graph
ingredient_graph = ingredient_df.withColumn('exploded',explode('ingredient_extract')) \
                      .select(col('title').alias('Recipe'),col('exploded').alias('Ingredient'))
#                       .withColumn("Frequency", lit(1))
ingredient_graph = ingredient_graph.filter(ingredient_graph.Ingredient != "")
ingredient_graph.show(20)


# graph_sample = ingredient_graph.toPandas()
# graph_sample.to_csv("graph_sample.csv")

##Let's try convert ingredient into count_frequency (BAD WAY, don't do)
# ingredient_freq = ingredient_graph.groupBy("Recipe").pivot("Ingredient").sum("Frequency")
# ingredient_freq.show()
# test = ingredient_freq.toPandas()
# test.to_csv("test.csv")
# print("test done")


##Proof of concept (Need to do !=)
# print(ingredient_graph.filter(ingredient_graph.Ingredient.isNull()).count())
# ingredient_graph = ingredient_graph.na.drop(subset=["Recipe"])
# test = ingredient_graph.filter(ingredient_graph.Recipe == "Grandma Emma's Spice Loaf")
# test.show()
# test2 = test.filter(ingredient_graph.Ingredient != "")
# test2.show()


In [ ]:
####----User Defined Function - Feature Engineering----####
##Labels: Vegetarian, Lactose, Nut, Seafood


#Safer to start off as non-vegetarian
def detectVege(li, vegDetect_list):
    label = 0
    detect_list = []
    for text in li:
        if text in vegDetect_list:
            detect_list.append(text)
    if (len(detect_list) == 0):
        label = 1
    return label


#Safer to start off as positive allergy
def detectNut(li):
    label = 1
    detect_list = []
    for text in li:
        if ("nut" in text):
            detect_list.append(text)
    if (len(detect_list) == 0):
        label = 0
    return label


def detectDairy(li):
    label = 1
    dairy_list = ["cheese", "milk", "yoghurt", "cream"]
    detect_list = []
    for text in li:
        if (text in dairy_list):
            detect_list.append(text)
    if (len(detect_list) == 0):
        label = 0
    return label


def detectSeafood(li, seaDetect_list):
    label = 1
    detect_list = []
    for text in li:
        if text in seaDetect_list:
            detect_list.append(text)
    if (len(detect_list) == 0):
        label = 0
    return label


##Read dictionary of food in food categories
with open("dict/meatDetect.txt", encoding="ISO-8859-1") as meat_file:
    vegDetect_list = meat_file.read().splitlines()

with open("dict/seafoodDetect.txt", encoding="ISO-8859-1") as seafood_file:
    seaDetect_list = seafood_file.read().splitlines()
    
    
##Create User Defined Function to detect and label food categories
udf_detectVege = udf(lambda x: detectVege(x, vegDetect_list), IntegerType())
udf_detectNut = udf(detectNut, IntegerType())
udf_detectDairy = udf(detectDairy, IntegerType())
udf_detectSeafood = udf(lambda x: detectSeafood(x, seaDetect_list), IntegerType())


##Labelling of ingredients
ingredient_label_df = ingredient_df.select("title", "ingredient_extract")
ingredient_label_df = ingredient_label_df.withColumn("vegetarian", udf_detectVege("ingredient_extract")) \
                           .withColumn("nut", udf_detectNut("ingredient_extract")) \
                           .withColumn("lactose", udf_detectDairy("ingredient_extract")) \
                           .withColumn("seafood", udf_detectSeafood("ingredient_extract"))

ingredient_label_df.show(5)


In [65]:
# test = labelling_df.toPandas()
# test.head()
#test.to_csv("test.csv")
# test = test.to_csv("test.csv")

In [ ]:
####----Convert Spark Data Frame to Pandas Data Frame----####
#ingredient_tbl_pdf = ingredient_graph.toPandas()
#ingredient_tbl_pdf


In [ ]:
####----Save Pandas Data Frame----####

# import re
# import time
# export_filename = 'recipe_allrecipes_' + (time.strftime('%Y%m%d-%H%M%S')) + '.json'
# ingredient_tbl_pdf.to_json(export_filename)

In [ ]:
####----Export Spark Data Frame----####

import re
import time
export_filename = 'recipe_allrecipes_' + (time.strftime('%Y%m%d-%H%M%S')) + '.json'
ingredient_df.write.save(export_filename)

In [ ]:
####----Upload file into HDFS----####
import os 
from subprocess import PIPE, Popen

##Create path to HDFS
hdfs_path = os.path.join(os.sep, 'user', 'centos', '/02_store/' + export_filename)

##Put files into HDFS
put_file = Popen(["hdfs", "dfs", "-put", export_filename, hdfs_path], stdin = PIPE, bufsize=-1)
put_file.communicate()

print('\nUpload completed and file stored in hdfs://localhost:9000' + hdfs_path)

In [ ]:
##Addition of other details
ingredient_tbl_df = ingredient_df.select("title", "rm_complete")
ingredient_tbl_df = ingredient_tbl_df.withColumn("Ing_Str", udf_convertString("rm_complete"))
# cleanIngredient_table = cleanIngredient_table.withColumn("Ing_Str", udf_convertString2("rm_complete"))
ingredient_tbl_df = ingredient_tbl_df.select("title", col('Ing_Str').alias('Ingredient'))
ingredient_tbl_df = ingredient_tbl_df.withColumn("ID", monotonically_increasing_id())

ingredient_tbl_df.show(5)
ingredient_tbl_df.count()

ingredient_tbl_pdf = ingredient_tbl_df.toPandas()
#ingredient_tbl_pdf.to_csv("RI.csv")

# cleanIngredient_table.createOrReplaceTempView('df')
# cleanIngredient_table = spark.sql('select row_number() over (order by "ID") as num, * from df')
# cleanIngredient_table.show(5)
# cleanIngredient_table.count()


In [ ]:
sim_mat = IndexedRowMatrix(sim_test).toBlockMatrix()
dot = sim_mat.multiply(sim_mat.transpose())
dot.toLocalMatrix().toArray()
dot_udf = udf(lambda x,y: float(x.dot(y)), DoubleType())

similarity.alias("i").join(similarity.alias("j"), col("i.ID") < col("j.ID"))\
    .select(
        col("i.ID").alias("i"), 
        col("j.ID").alias("j"), 
        dot_udf("i.ID", "j.ID").alias("cosine"))\
    .sort("i", "j")\
    .show()
